### Data Normalization

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional

import torchvision 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display,clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import OrderedDict
from collections import namedtuple

In [2]:
train_set=torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]))

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)
        
    def forward(self,t):
        # layer1-input layer
        t=t
        
        # layer2-hidden conv layer        
        t=self.conv1(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer3-hidden conv layer        
        t=self.conv2(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer4-hidden linear layer        
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=functional.relu(t)
        
        # layer5-hidden linear layer        
        t=self.fc2(t)
        t=functional.relu(t)
        
        # layer6-output layer        
        t=self.out(t)
        
        return t

In [4]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run=namedtuple("Run",params.keys())
        
        runs=[]
        for v in product(*params.values()):
            runs.append(Run(*v))
        
        return runs

In [5]:
class RunManager():
    def __init__(self):
        self.epoch_count=0
        self.epoch_loss=0
        self.epoch_num_correct=0
        self.epoch_start_time=None
        
        self.run_params=None
        self.run_count=0
        self.run_data=[]
        self.run_start_time=None
        
        self.network=None
        self.loader=None
        self.tb=None
        
    def begin_run(self,run,network,loader):
        self.run_start_time=time.time()
        self.run_params=run
        self.run_count+=1
        
        self.network=network
        self.loader=loader
        self.tb=SummaryWriter(comment=f"-{run}")
        
        images,labels=next(iter(self.loader))
        grid=torchvision.utils.make_grid(images)
        
        self.tb.add_image("images",grid)
        self.tb.add_graph(self.network,images.to(getattr(run,'device','cpu')))
        
    def end_run(self):
        self.tb.close()
        self.epoch_count=0
        
    def begin_epoch(self):
        self.epoch_start_time=time.time()
        self.epoch_count+=1
        self.epoch_loss=0
        self.epoch_num_correct=0
        
    def end_epoch(self):
        epoch_duration=time.time()-self.epoch_start_time
        run_duration=time.time()-self.run_start_time
        
        loss=self.epoch_loss/len(self.loader.dataset)
        accuracy=self.epoch_num_correct/len(self.loader.dataset)
        
        self.tb.add_scalar("Loss",loss,self.epoch_count)
        self.tb.add_scalar("Accuracy",accuracy,self.epoch_count)
        
        for name,param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f"{name}.grad",param.grad,self.epoch_count)
        
        results=OrderedDict()
        results["run"]=self.run_count
        results["epoch"]=self.epoch_count
        results["loss"]=loss
        results["accuracy"]=accuracy
        results["epoch duration"]=epoch_duration
        results["run duration"]=run_duration
        
        for k,v in self.run_params._asdict().items(): results[k]=v
        self.run_data.append(results)
        
        df=pd.DataFrame.from_dict(self.run_data,orient="columns")
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self,loss,batch):
        self.epoch_loss+=loss.item()*batch[0].shape[0]

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct+=self._get_num_correct(preds,labels)

    @torch.no_grad()
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def save(self,filename):
        pd.DataFrame.from_dict(self.run_data,orient="columns").to_csv(f"{filename}.csv")

        with open(f"{filename}.json","w",encoding="utf-8") as f:
            json.dump(self.run_data,f,ensure_ascii=False,indent=4)
            

#### Method 1

In [6]:
loader=DataLoader(train_set,batch_size=len(train_set),num_workers=1)
data=next(iter(loader))
data[0].mean(),data[0].std()

(tensor(0.2861), tensor(0.3530))

In [7]:
loader=DataLoader(train_set,batch_size=1000,num_workers=1)
num_of_pixels=len(train_set)*28*28

total_sum=0
for batch in loader:
    total_sum+=batch[0].sum()
mean=total_sum/num_of_pixels

sum_of_squared_error=0
for batch in loader:
    sum_of_squared_error+=((batch[0]-mean).pow(2)).sum()
std=torch.sqrt(sum_of_squared_error/num_of_pixels)

mean,std

(tensor(0.2860), tensor(0.3530))

#### Plotting Values

In [8]:
# plt.hist(data[0].flatten())
# plt.axvline(data[0].mean())

### Using mean and std values

In [9]:
train_set_normal=torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)]))

In [10]:
loader=DataLoader(train_set_normal,batch_size=len(train_set),num_workers=1)
data=next(iter(loader))
data[0].mean(),data[0].std()

(tensor(-6.7840e-06), tensor(1.0000))

In [11]:
# plt.hist(data[0].flatten())
# plt.axvline(data[0].mean())

In [12]:
trainsets={'not_normal':train_set,
          'normal':train_set_normal}

In [13]:
params=OrderedDict( lr=[0.01],
                    batch_size=[1000],
                    num_workers=[1],
                    device=['cuda'],
                  trainset=['not_normal','normal'])


m=RunManager()
for run in RunBuilder.get_runs(params):
    device=torch.device(run.device)
    network=Network().to(device)
    loader=DataLoader(trainsets[run.trainset],batch_size=run.batch_size,num_workers=run.num_workers)
    optimizer=optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    for epoch in range(20):
        m.begin_epoch()
        for batch in loader:
            images=batch[0].to(device)
            labels=batch[1].to(device)
            preds=network(images)
            loss=functional.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss,batch)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
0,1,1,1.008232,0.614567,5.939913,6.991661,0.01,1000,1,cuda,not_normal
1,1,2,0.561127,0.782583,6.092527,13.223632,0.01,1000,1,cuda,not_normal
2,1,3,0.476935,0.821617,5.663989,18.985905,0.01,1000,1,cuda,not_normal
3,1,4,0.427782,0.842483,5.605058,24.686243,0.01,1000,1,cuda,not_normal
4,1,5,0.396422,0.853250,6.071294,30.857231,0.01,1000,1,cuda,not_normal
5,1,6,0.374200,0.862550,6.087299,37.100949,0.01,1000,1,cuda,not_normal
6,1,7,0.354540,0.868317,6.091551,43.344014,0.01,1000,1,cuda,not_normal
7,1,8,0.344047,0.872050,5.845151,49.327017,0.01,1000,1,cuda,not_normal
8,1,9,0.333103,0.876317,5.926697,55.388990,0.01,1000,1,cuda,not_normal
9,1,10,0.322157,0.880517,5.868866,61.373769,0.01,1000,1,cuda,not_normal


In [14]:
pd.DataFrame.from_dict(m.run_data).sort_values('accuracy',ascending=False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
39,2,20,0.221373,0.916050,9.398087,184.176079,0.01,1000,1,cuda,normal
38,2,19,0.225781,0.914133,9.308795,174.649629,0.01,1000,1,cuda,normal
35,2,16,0.235383,0.912283,9.029200,146.341507,0.01,1000,1,cuda,normal
36,2,17,0.235896,0.910583,9.350322,155.825292,0.01,1000,1,cuda,normal
34,2,15,0.240785,0.910200,8.927573,137.180276,0.01,1000,1,cuda,normal
33,2,14,0.242585,0.910000,9.209831,128.115682,0.01,1000,1,cuda,normal
37,2,18,0.237166,0.909783,9.279850,165.202118,0.01,1000,1,cuda,normal
32,2,13,0.241946,0.909133,8.953985,118.783297,0.01,1000,1,cuda,normal
31,2,12,0.246658,0.907917,9.183056,109.670047,0.01,1000,1,cuda,normal
30,2,11,0.257177,0.903867,9.442148,100.356219,0.01,1000,1,cuda,normal
